<a href="https://colab.research.google.com/github/thainazanfolin/great_expectations_tests/blob/main/great_expectations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install great_expectations

imports

In [4]:
import kagglehub
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import great_expectations as ge

## 1. Carregando o Dataset
Carregando o dataset a partir do arquivo raw no github.

In [5]:

url = 'https://raw.githubusercontent.com/thainazanfolin/great_expectations_tests/refs/heads/main/disney_princess_popularity_dataset_300_rows.csv'
df = pd.read_csv(url)

# visão inicial do df
df.head()



,PrincessName,FirstMovieTitle,FirstMovieYear,MovieRuntimeMinutes,NumberOfSongs,HasSoloSong,HasDuet,HairColor,EyeColor,OutfitPrimaryColor,...,RottenTomatoesScore,BoxOfficeMillions,AvgScreenTimeMinutes,NumMerchItemsOnAmazon,GoogleSearchIndex2024,InstagramFanPages,TikTokHashtagViewsMillions,Top3Hashtags,IsIconic,PopularityScore
0,Pocahontas,Pocahontas's Adventure,1977,85,4,No,Yes,Brown,Green,Pink,...,88,661,25,2524,68,298,481,#TeamPocahontas #LovePocahontas #PocahontasFor...,Yes,88
1,Mulan,Mulan's Adventure,1995,85,1,Yes,Yes,Blonde,Brown,Purple,...,91,688,52,4202,49,133,619,#TeamMulan #LoveMulan #MulanForever,Yes,78
2,Raya,Raya's Adventure,1989,110,4,No,No,Blonde,Hazel,Pink,...,76,790,54,525,68,129,230,#TeamRaya #LoveRaya #RayaForever,No,53
3,Anna,Anna's Adventure,1940,78,1,No,No,Brown,Grey,Green,...,70,820,42,1951,73,138,697,#TeamAnna #LoveAnna #AnnaForever,Yes,84
4,Mulan,Mulan's Adventure,1943,103,7,No,Yes,White,Brown,Green,...,86,996,55,3958,34,155,200,#TeamMulan #LoveMulan #MulanForever,Yes,71


## 2. Análise exploratória dos dados
Iniciando a análise exploratória para entender possíveis testes úteis no momento do ETL.  

In [6]:
# número de linhas e colunas
print(f"\nLinhas e colunas: {df.shape[0]} linhas e {df.shape[1]} colunas")

# nomes das colunas
print("\nColunas:")
print(df.columns.tolist())

# tipos de dados e contagem (não nulos)
print("\nInfos:")
print(df.info())

# valores nulos por coluna
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# % de valores nulos
print("\n% de valores nulos por coluna:")
print((df.isnull().mean() * 100).round(2))

# Colunas numéricas
print("\nEstatísticas descritivas (colunas numéricas):")
print(df.describe())

# Colunas categóricas
print("\nEstatísticas descritivas (colunas categóricas):")
print(df.describe(include=['object']))

# Valores únicos por coluna (motivo:útil para saber se precisa validar um domínio fixo)
print("\nValores únicos por coluna categórica:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nColuna '{col}' - {df[col].nunique()} valores únicos:")
    print(df[col].unique())

# Detecta possíveis colunas com valores duplicados
print("\nRegistros duplicados (linhas inteiras):", df.duplicated().sum())

# Verifica se há colunas que parecem ser identificadores únicos
print("\nColunas com valores únicos em todas as linhas (possíveis IDs):")
for col in df.columns:
    if df[col].is_unique:
        print(f"- {col}")



Linhas e colunas: 300 linhas e 36 colunas

Colunas:
['PrincessName', 'FirstMovieTitle', 'FirstMovieYear', 'MovieRuntimeMinutes', 'NumberOfSongs', 'HasSoloSong', 'HasDuet', 'HairColor', 'EyeColor', 'OutfitPrimaryColor', 'OutfitStyleEra', 'IsRoyalByBirth', 'HasAnimalSidekick', 'SidekickType', 'HasMagicalPowers', 'MagicType', 'MainSetting', 'IsBasedOnRealStory', 'CulturalOrigin', 'SpeaksToAnimals', 'FightsVillainDirectly', 'RomanticSubplot', 'MarriedByEnd', 'VillainName', 'VillainType', 'IMDB_Rating', 'RottenTomatoesScore', 'BoxOfficeMillions', 'AvgScreenTimeMinutes', 'NumMerchItemsOnAmazon', 'GoogleSearchIndex2024', 'InstagramFanPages', 'TikTokHashtagViewsMillions', 'Top3Hashtags', 'IsIconic', 'PopularityScore']

Infos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PrincessName                300 non-null    obj

# **Resultado da análise exploratória**
* dataset possui 300 linhas e 36 colunas.
* temos 24 colunas do tipo object (categóricas ou textuais), 11 colunas do tipo int64 (numéricas inteiras) e 1 do tipo float64 (IMDB_Rating).

Há valores nulos nas seguintes colunas:
* SidekickType: 23,33% de valores ausentes
* MagicType: 17,67% de valores ausentes
* VillainType: 15,67% de valores ausentes

### **IMPORTANTE!**
Essas colunas precisarão de tratamento específico, como imputação, categorização de ausentes ou exclusão.

A análise exploratória também revela a importância de verificar:

Colunas booleanas codificadas como object **precisarão ser convertidas em booleanas reais para facilitar as expectativas com Great Expectations.**

## **Usando o Great Expectations para ETL**

### O que vamos fazer com o great_expectations:

1. verificar a distribuição de valores únicos em colunas categóricas (como HairColor, OutfitStyleEra, CulturalOrigin, VillainType), que serão úteis para aplicar expectativas como expect_column_values_to_be_in_set.

2. Consistência entre colunas relacionadas:

* Se HasAnimalSidekick for False, SidekickType deve ser nulo

* Se HasMagicalPowers for False, MagicType deve ser nulo

* Se IsBasedOnRealStory for True, talvez MagicType devesse ser nulo (ou investigar exceções)

3. A necessidade de expectativas relacionadas à completude dos dados em colunas críticas, como expect_column_values_to_not_be_null

4. Detecção de outliers em colunas como TikTokHashtagViewsMillions, BoxOfficeMillions e PopularityScore

5. Relações de correlação entre PopularityScore e métricas como InstagramFanPages, IMDB_Rating, e GoogleSearchIndex2024, que podem embasar regras para detecção de anomalias futuras


# Instalando Great_Expectations

In [7]:
!pip install great_expectations==0.17.21
import great_expectations as ge

# contexto
context = ge.get_context()

# Adiciona um datasource do tipo pandas
datasource = context.sources.add_pandas(name="datasource")

# Adiciona o asset com o DataFrame
asset = datasource.add_dataframe_asset(name="DisneyPrincess", dataframe=df)

# Cria o batch request
batch_request = asset.build_batch_request()


INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpwcvw6zte' for ephemeral docs site


In [8]:
# Colunas que NÃO PODEM TER VALORES NULOS
expect_column_values_to_not_be_null('PrincessName')
expect_column_values_to_not_be_null('PopularityScore')
expect_column_values_to_not_be_null('BoxOfficeRevenue')

NameError: name 'expect_column_values_to_not_be_null' is not defined